<a href="https://colab.research.google.com/github/jyryu3161/2023ksbb/blob/main/2023KSBB_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESM embedding 예

In [2]:
!pip install git+https://github.com/facebookresearch/esm.git

  Cloning https://github.com/facebookresearch/esm.git to /tmp/pip-req-build-8a99fncj
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/esm.git /tmp/pip-req-build-8a99fncj
  Resolved https://github.com/facebookresearch/esm.git to commit 2b369911bb5b4b0dda914521b9475cad1656b2ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import joblib
import sys
import numpy as np
import torch
import esm

sys.modules['sklearn.externals.joblib'] = joblib

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

seq_info = {}
seq_info['Seq1'] = ['AVIYYRFRSQKPDHIATIKFDGTGLTVFELKRDIILANNLLHSTDV', 1]
seq_info['Seq2'] = ['WGYQNGGSSSAGERELDDDNEVVPRSTTVLVRRTMTPKKN', 0]
seq_info['Seq3'] = ['KGNVQRYVAGKPRLQVSGTNSVNKSISLGNNVGGTMNFGDAATNGDEDDMIKKMF', 1]

X = []
y = []
seq_id_list = []

for each_seq_id in seq_info:
    label = seq_info[each_seq_id][1]
    seq = seq_info[each_seq_id][0]
    data = [
        (each_seq_id, seq),
    ]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33].detach().cpu().numpy()

    sequence_representations = []
    for i, (_, seq) in enumerate(data):
        sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

    tmp_feature = sequence_representations[0]
    seq_id_list.append(each_seq_id)
    X.append(tmp_feature)
    y.append(each_seq_id) # label

X = np.asarray(X)
y = np.asarray(y)
seq_id_list = np.asarray(seq_id_list)

print(X.shape)
print(y.shape)
print(seq_id_list.shape)

print(X)

(3, 1280)
(3,)
(3,)
[[ 0.03685372  0.04412491 -0.01071034 ... -0.11547086 -0.01925141
   0.03808342]
 [ 0.02888836 -0.04779714  0.05215322 ...  0.03859621 -0.07909244
   0.07299711]
 [-0.01755035 -0.02575365  0.00276981 ... -0.0132841  -0.10283709
   0.06478625]]


# EvoDiff 기반 서열 생성 예

In [16]:
import sys
!{sys.executable} -m pip install evodiff
!pip install torch_geometric
!pip install torch-scatter


fatal: destination path '2023ksbb' already exists and is not an empty directory.


In [24]:
from evodiff.pretrained import MSA_OA_DM_MAXSUB

checkpoint = MSA_OA_DM_MAXSUB()
model, collater, tokenizer, scheme = checkpoint

from evodiff.generate_msa import generate_query_oadm_msa_simple
import re

path_to_msa = './2023ksbb/bfd_uniclust_hits.a3m'
n_sequences=32 # number of sequences in MSA to subsample
seq_length=256 # maximum sequence length to subsample
selection_type='random' # or 'MaxHamming'; MSA subsampling scheme

tokeinzed_sample, generated_sequence  = generate_query_oadm_msa_simple(path_to_msa, model, tokenizer, n_sequences, seq_length, device='cpu', selection_type=selection_type)

print("New sequence (no gaps, pad tokens)", re.sub('[!-]', '', generated_sequence[0][0],))


 21%|██▏       | 35/164 [27:56<1:42:59, 47.91s/it]


KeyboardInterrupt: ignored